# MNIST Classifier from scratch

In [1]:
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import fastbook
from fastai.vision.all import *
from fastbook import *

## The MNIST Dataset

First, we import the full MNIST Dataset

In [2]:
path = untar_data(URLs.MNIST)
Path.BASE_PATH = path

In [4]:
print(path.ls(), (path/'training').ls())

[Path('training'), Path('testing')] [Path('training/4'), Path('training/3'), Path('training/5'), Path('training/1'), Path('training/7'), Path('training/6'), Path('training/8'), Path('training/0'), Path('training/9'), Path('training/2')]


Now we want to create a tensor from each image in the dataset, and have them all appropiately labeled. For that reason, we'll create a dictionary and assign each number as a key to a list of the tensors of each class.

In [5]:
digits = {}
for i in range(10):
  digits[i] = (path/'training'/str(i)).ls().sorted()

tensors = [[tensor(Image.open(o)) for o in digits[i]] for i in range(10)]

In [6]:
img0_path = digits[2][1040]
im0 = Image.open(img0_path)
im0

Some validations!
- Of the size of each class

In [7]:
for i in range(10):
  print('{}: '.format(i), len(tensors[i]))

0:  5923
1:  6742
2:  5958
3:  6131
4:  5842
5:  5421
6:  5918
7:  6265
8:  5851
9:  5949


- Of the shape of each tensor

In [8]:
tensors[3][0].shape

torch.Size([28, 28])

Now we want to stack each class into a rank-3 tensor:

In [9]:
stacked = [torch.stack(tensors[i]).float()/255 for i in range(10)]
for i in range(10):
  print('{}: '.format(i), stacked[i].shape)

0:  torch.Size([5923, 28, 28])
1:  torch.Size([6742, 28, 28])
2:  torch.Size([5958, 28, 28])
3:  torch.Size([6131, 28, 28])
4:  torch.Size([5842, 28, 28])
5:  torch.Size([5421, 28, 28])
6:  torch.Size([5918, 28, 28])
7:  torch.Size([6265, 28, 28])
8:  torch.Size([5851, 28, 28])
9:  torch.Size([5949, 28, 28])


In [10]:
valid_digits = {}
for i in range(10):
  valid_digits[i] = (path/'testing'/str(i)).ls().sorted()

valid_tensors = [[tensor(Image.open(o)) for o in valid_digits[i]] for i in range(10)]
valid_stacked = [torch.stack(valid_tensors[i]).float()/255 for i in range(10)]

for i in range(10):
  print('valid_{}: '.format(i), valid_stacked[i].shape)

valid_0:  torch.Size([980, 28, 28])
valid_1:  torch.Size([1135, 28, 28])
valid_2:  torch.Size([1032, 28, 28])
valid_3:  torch.Size([1010, 28, 28])
valid_4:  torch.Size([982, 28, 28])
valid_5:  torch.Size([892, 28, 28])
valid_6:  torch.Size([958, 28, 28])
valid_7:  torch.Size([1028, 28, 28])
valid_8:  torch.Size([974, 28, 28])
valid_9:  torch.Size([1009, 28, 28])


Finally, for the training set, we want to create a single rank-2 tensor containing all classes stacked and every image transformed to a vector. For the validating set, we want a hot-encoded vector for each integer the image represents.

In [11]:
train_x = torch.cat([stacked[i] for i in range(10)]).view(-1,28*28)
train_y = []
for i in range(10):
    zeros = [0]*tensor(10)
    zeros[i] = 1.
    train_y += zeros * len(digits[i])
train_y = tensor(train_y).view((-1,10))
train_x.shape, train_y.shape

(torch.Size([60000, 784]), torch.Size([60000, 10]))

In [12]:
dset = list(zip(train_x, train_y))

Doing the same for the vaildation set

In [13]:
valid_x = torch.cat([valid_stacked[i] for i in range(10)]).view(-1,28*28)
valid_y = []
for i in range(10):
    zeros = [0]*tensor(10)
    zeros[i] = 1.
    valid_y += zeros * len(valid_digits[i])
valid_y = tensor(valid_y).view((-1,10))
valid_x.shape, valid_y.shape

(torch.Size([10000, 784]), torch.Size([10000, 10]))

In [14]:
x,y = dset[9]; x.shape, y

(torch.Size([784]), tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [15]:
valid_dset = list(zip(valid_x, valid_y))

## Loss function

In [16]:
def init_params(size, std=1): return (torch.randn(size)*std).requires_grad_()
weights = init_params((28*28,10))
bias = init_params(1)

In [17]:
def linear1(xb): return F.softmax(xb@weights + bias, dim=1)
preds = linear1(train_x)
preds

tensor([[4.6670e-12, 6.0144e-08, 1.9665e-04,  ..., 5.9243e-07, 1.1039e-06, 9.9382e-01],
        [3.3582e-09, 1.7959e-07, 2.8649e-05,  ..., 1.0690e-09, 3.9134e-05, 1.0747e-01],
        [6.2731e-22, 2.6699e-10, 8.1824e-07,  ..., 2.4047e-12, 9.5857e-01, 4.1432e-02],
        ...,
        [1.2629e-13, 7.4681e-04, 2.9012e-04,  ..., 3.5388e-05, 6.4642e-02, 9.3425e-01],
        [3.0635e-14, 6.9778e-09, 6.0548e-07,  ..., 6.8531e-07, 9.9697e-01, 2.1861e-03],
        [1.1982e-10, 5.9056e-03, 7.6727e-03,  ..., 7.2289e-01, 1.4260e-03, 2.4199e-01]], grad_fn=<SoftmaxBackward0>)

In [18]:
def mnist_loss(predictions, targets):
  predictions = predictions.softmax(1)
  return torch.where(targets==1,1-predictions,predictions).mean()

In [19]:
dl = DataLoader(dset, batch_size=256)
xb, yb = first(dl)
xb.shape, yb.shape

(torch.Size([256, 784]), torch.Size([256, 10]))

In [20]:
valid_dl = DataLoader(valid_dset, batch_size = 256)

### Sidebar: Tests

In [21]:
# hide
## Creating a mini-batch of size 5 for testing
batch = train_x[:5]
batch.shape # Outputs torch.Size([5, 784])

torch.Size([5, 784])

In [22]:
preds = linear1(batch)
loss = mnist_loss(preds, train_y[:5])
loss

tensor(0.1827, grad_fn=<MeanBackward0>)

In [23]:
# Calculating the gradients
loss.backward()
weights.grad.shape,weights.grad.mean(),bias.grad # Outputs (torch.Size([784, 1]), tensor(-0.0001), tensor([-0.0008]))

(torch.Size([784, 10]), tensor(-3.3558e-12), tensor([-2.1100e-10]))

### End sidebar

In [24]:
def calc_grad(xb, yb, model):
  preds = model(xb)
  loss = mnist_loss(preds, yb)
  loss.backward()

In [25]:
def train_epoch(model, lr, params):
  for xb,yb in dl:
    calc_grad(xb, yb, model)
    for p in params:
      p.data -= p.grad*lr 
      p.grad.zero_()

def batch_accuracy(xb, yb):
    preds = xb.argmax(dim=1)
    true_classes = yb.argmax(dim=1)
    # correct = preds == yb.squeeze()
    # return correct.float().mean()
    correct = (preds == true_classes).sum().item()
    total = xb.size(0)
    return tensor(correct / total)

In [26]:
batch_accuracy(linear1(batch), train_y[:5])

tensor(0.)

In [27]:
def validate_epoch(model):
  accs = [batch_accuracy(model(xb), yb) for xb,yb in valid_dl]
  return round(torch.stack(accs).mean().item(),4)

In [28]:
validate_epoch(linear1)

0.1253

In [29]:
lr = 1.
params = weights,bias
train_epoch(linear1, lr, params)
validate_epoch

<function __main__.validate_epoch(model)>

In [30]:
for i in range(20):
  train_epoch(linear1, lr, params)
  print(validate_epoch(linear1), end=' ')

0.1467 0.1561 0.1688 0.1811 0.1923 0.2051 0.2201 0.2297 0.2393 0.2501 0.2602 0.2716 0.2823 0.2925 0.3037 0.3123 0.3205 0.3284 0.3359 0.3418 

## Creating an Optimizer

In [31]:
linear_model = nn.Linear(28*28,10)
w,b = linear_model.parameters()
w.shape,b.shape

(torch.Size([10, 784]), torch.Size([10]))

In [32]:
class BasicOptim:
  def __init__(self,params,lr): self.params,self.lr = list(params), lr
  
  def step(self, *args, **kwargs):
    for p in self.params: p.data -= p.grad.data * self.lr

  def zero_grad(self, *args, **kwargs):
    for p in self.params: p.grad = None

opt = BasicOptim(linear_model.parameters(), lr)

def train_epoch(model):
  for xb,yb in dl:
    calc_grad(xb,yb,model)
    opt.step()
    opt.zero_grad()

In [33]:
validate_epoch(linear_model)

0.065

In [34]:
def train_model(model, epochs):
  for i in range(epochs):
    train_epoch(model)
    print(validate_epoch(model), end=' ')

In [35]:
train_model(linear_model, 20)

0.2261 0.2646 0.3722 0.4772 0.488 0.5294 0.5718 0.478 0.4441 0.4836 0.5183 0.5476 0.572 0.5912 0.6075 0.622 0.6334 0.6443 0.6554 0.6662 

In [36]:
dls = DataLoaders(dl, valid_dl)
learn = Learner(dls, nn.Linear(28*28,10), opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [37]:
learn.fit(20,lr=lr)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.180068,0.161426,0.197600,00:00
1,0.152673,0.141732,0.322100,00:00
2,0.143203,0.127619,0.384800,00:00
3,0.137055,0.115690,0.459600,00:00
4,0.131596,0.106320,0.503500,00:00
5,0.130316,0.098618,0.542700,00:00
6,0.129861,0.092294,0.579200,00:00
7,0.129627,0.086995,0.603200,00:00
8,0.129475,0.082605,0.624700,00:00
9,0.129364,0.078952,0.644600,00:00


## Adding a non-linearity

In [39]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,10)
)

In [40]:
learn = Learner(dls, simple_net, opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [41]:
learn.fit(40,0.1)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.181002,0.177281,0.142800,00:01
1,0.178668,0.171460,0.373900,00:00
2,0.173569,0.161231,0.394700,00:00
3,0.156330,0.151298,0.331300,00:01
4,0.128234,0.145241,0.327900,00:01
5,0.117303,0.133916,0.384400,00:01
6,0.107615,0.124814,0.418500,00:01
7,0.099337,0.117569,0.446700,00:01
8,0.094120,0.111720,0.476100,00:01
9,0.090793,0.107077,0.497700,00:00


In [10]:
dls = ImageDataLoaders.from_folder(path, train='training', valid='testing', device='mps')
learn = vision_learner(dls, resnet18, pretrained=False, loss_func=F.cross_entropy, metrics=accuracy)

In [11]:
learn.fit_one_cycle(1,0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.095913,0.039997,0.987200,01:07
